In [ ]:
# Use Openstreet map to find AWO facilities

In [1]:
import requests
import json

In [ ]:
overpass_url = "https://overpass-api.de/api/interpreter"

In [5]:
query = """
[out:json][timeout:30];
area["ISO3166-1"="DE"][admin_level=2]->.a;
(
    nwr["name"~"AWO|Arbeiterwohlfahrt", i](area.a);
    nwr["brand"~"AWO|Arbeiterwohlfahrt", i](area.a);
    nwr["operator"~"AWO|Arbeiterwohlfahrt", i](area.a);

);
out center;
"""

In [2]:
query = """
[out:json][timeout:30];
area["ISO3166-1"="DE"]->.a;
(
  relation["boundary"="administrative"]["admin_level"="4"](area.a);
);
out body;
>;
out skel qt;
"""

response = requests.post(overpass_url, data={'data': query})
data = response.json()

# Print the names and IDs of the states
for element in data['elements']:
    if 'tags' in element:
        tags = element['tags']
        name = tags.get('name', 'N/A')
        official_name = tags.get('official_name', 'N/A')
        print(f"Name: {name}, Official Name: {official_name}, Relation ID: {element['id']}")

Name: Mecklenburg-Vorpommern, Official Name: N/A, Relation ID: 28322
Name: Groningen, Official Name: Groningen, Relation ID: 47826
Name: Schleswig-Holstein, Official Name: N/A, Relation ID: 51529
Name: Rheinland-Pfalz, Official Name: N/A, Relation ID: 62341
Name: Thüringen, Official Name: Freistaat Thüringen, Relation ID: 62366
Name: Saarland, Official Name: N/A, Relation ID: 62372
Name: Berlin, Official Name: Berlin, Relation ID: 62422
Name: Sachsen, Official Name: Freistaat Sachsen, Relation ID: 62467
Name: Brandenburg, Official Name: N/A, Relation ID: 62504
Name: Sachsen-Anhalt, Official Name: N/A, Relation ID: 62607
Name: Baden-Württemberg, Official Name: N/A, Relation ID: 62611
Name: Hessen, Official Name: N/A, Relation ID: 62650
Name: Bremen, Official Name: Freie Hansestadt Bremen, Relation ID: 62718
Name: Nordrhein-Westfalen, Official Name: N/A, Relation ID: 62761
Name: Niedersachsen, Official Name: N/A, Relation ID: 62771
Name: Hamburg, Official Name: Freie und Hansestadt Hambu

In [11]:
state_dict = {}
for element in data['elements']:
    if 'tags' in element:
        tags = element['tags']
        name = tags.get('name', 'N/A')
        iso_name = tags.get('ISO3166-2')
        if "DE" not in iso_name:
            continue
        state_dict[name] = iso_name
        print(f"Name: {name}, ISO Name: {iso_name}")

Name: Mecklenburg-Vorpommern, ISO Name: DE-MV
Name: Schleswig-Holstein, ISO Name: DE-SH
Name: Rheinland-Pfalz, ISO Name: DE-RP
Name: Thüringen, ISO Name: DE-TH
Name: Saarland, ISO Name: DE-SL
Name: Berlin, ISO Name: DE-BE
Name: Sachsen, ISO Name: DE-SN
Name: Brandenburg, ISO Name: DE-BB
Name: Sachsen-Anhalt, ISO Name: DE-ST
Name: Baden-Württemberg, ISO Name: DE-BW
Name: Hessen, ISO Name: DE-HE
Name: Bremen, ISO Name: DE-HB
Name: Nordrhein-Westfalen, ISO Name: DE-NW
Name: Niedersachsen, ISO Name: DE-NI
Name: Hamburg, ISO Name: DE-HH
Name: Hamburg, ISO Name: DE-HH
Name: Bayern, ISO Name: DE-BY


In [19]:
query_search_awo = f"""
    [out:json][timeout:200];
    area["name"="Berlin"]->.searchArea;
    (
    nwr["name"~"AWO|Arbeiterwohlfahrt", i](area.searchArea);
    nwr["brand"~"AWO|Arbeiterwohlfahrt", i](area.searchArea);
    nwr["operator"~"AWO|Arbeiterwohlfahrt", i](area.searchArea);
    );
    out center;
    """
response = requests.post(overpass_url, data={'data': query_search_awo})
data = response.json()

In [22]:
len(data['elements'])

84

In [23]:
all_awo_elements = []

# Now, loop through each state and search for AWO facilities
for key in state_dict.keys():
    state_id = state_dict[key]
    print(f"Querying state with relation ID: {state_id}")

    query_search_awo = f"""
    [out:json][timeout:200];
    area["ISO3166-2"="{state_id}"]->.searchArea;
    (
    nwr["name"~"AWO|Arbeiterwohlfahrt", i](area.searchArea);
    nwr["brand"~"AWO|Arbeiterwohlfahrt", i](area.searchArea);
    nwr["operator"~"AWO|Arbeiterwohlfahrt", i](area.searchArea);
    );
    out center;
    """

    try:
        response = requests.post(overpass_url, data={'data': query_search_awo})
        data = response.json()
        all_awo_elements.extend(data['elements'])
        print(f"   Found {len(data['elements'])} AWO facilities.")
    except Exception as e:
        print(f"   Error querying state {state_id}: {e}")

print(f"\nTotal AWO facilities found across all states: {len(all_awo_elements)}")

# Process your results (all_awo_elements) here
# For example, you can save them to a file:
with open('awo_facilities_germany.json', 'w', encoding='utf-8') as f:
    json.dump(all_awo_elements, f, ensure_ascii=False, indent=2)

Querying state with relation ID: DE-MV
   Found 156 AWO facilities.
Querying state with relation ID: DE-SH
   Found 189 AWO facilities.
Querying state with relation ID: DE-RP
   Found 94 AWO facilities.
Querying state with relation ID: DE-TH
   Found 300 AWO facilities.
Querying state with relation ID: DE-SL
   Found 121 AWO facilities.
Querying state with relation ID: DE-BE
   Error querying state DE-BE: Expecting value: line 1 column 1 (char 0)
Querying state with relation ID: DE-SN
   Found 281 AWO facilities.
Querying state with relation ID: DE-BB
   Found 163 AWO facilities.
Querying state with relation ID: DE-ST
   Found 142 AWO facilities.
Querying state with relation ID: DE-BW
   Found 299 AWO facilities.
Querying state with relation ID: DE-HE
   Found 237 AWO facilities.
Querying state with relation ID: DE-HB
   Found 69 AWO facilities.
Querying state with relation ID: DE-NW
   Error querying state DE-NW: Expecting value: line 1 column 1 (char 0)
Querying state with relation I